In [181]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

##### Club name and id

##### Funcs

In [219]:
"""
To make the request to the page we have to inform the
website that we are a browser and that is why we
use the headers variable
"""
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

def creating_club_dataframe(
        club_name, 
        club_id, 
        headers
    ):

    # endereco_da_pagina stands for the data page address
    endereco_da_pagina = "https://www.transfermarkt.com.br/"+club_name+"/startseite/verein/"+club_id+"?saison_id=2021"

    # In the objeto_response variable we will the download of the web page
    objeto_response = requests.get(endereco_da_pagina, headers=headers)

    """
    Now we will create a BeautifulSoup object from our object_response.
    The 'html.parser' parameter represents which parser we will use when creating our object,
    a parser is a software responsible for converting an entry to a data structure.
    """
    pagina_bs = BeautifulSoup(objeto_response.content, 'html.parser')

    club_jog = [] # List that will receive all the players club - 

    transfmkt_site = 'https://www.transfermarkt.com.br'

    # Players names
    nomes_jog = [] # List that will receive all the players names - check
    tags_nomes = pagina_bs.find_all("span", {"class": "show-for-small"})
    for tag_nome in tags_nomes:
        nomes_jog.append(tag_nome.text)

    # Players links
    links_jog = [] # List that will receive all the players profile links - check
    tags_links = BeautifulSoup(str(tags_nomes).strip('[]'), 'html.parser')
    for a in tags_links.find_all('a', href=True):
        links_jog.append(a['href'])

    # Players clubs
    club_name_of = club_name.replace("-","_")
    club_jog = [] # List that will receive all the players club - check
    for c in range(len(nomes_jog)):
        club_jog.append(club_name_of)

    # Clubs championship
    camp_jog = []
    tags_camp = pagina_bs.find_all("span", {"class": "hauptpunkt"})
    for tag_camp in tags_camp:
        camp = str(tag_camp.text).replace(" ", "").replace("\n","")

    for c in range(len(nomes_jog)):
        camp_jog.append(camp)

    # Players Positions
    posicoes_jog = [] # List that will receive all the players positions - check
    tags_goleiros = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Torwart"})
    tags_zagueiros = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Abwehr"})
    tags_meias = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Mittelfeld"})
    tags_atacantes = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Sturm"})

    # Gk positions
    for c in range(len(tags_goleiros)):
        posicoes_jog.append("Goleiro")

    # Defenders positions
    for c in range(len(tags_zagueiros)):
        posicoes_jog.append("Defensor")

    # Midfielders positions
    for c in range(len(tags_meias)):
        posicoes_jog.append("Meia")

    # Strikers positions
    for c in range(len(tags_atacantes)):
        posicoes_jog.append("Atacante")

    # Players age, heights, nacionalities
    idade_jog = [] # List that will receive all the players age - check
    alt_jog = [] # List that will receive all the players height - check
    nac_jog = [] # List that will receive all the players nacionalities - check

    for link in links_jog:
        end = transfmkt_site + link
        obj_resp = requests.get(end, headers=headers)
        pag_bs = BeautifulSoup(obj_resp.content, 'html.parser')
        tags_idades = pag_bs.find_all("span", {"itemprop": "birthDate"})
        tags_nac = pag_bs.find_all("span", {"itemprop": "nationality"})
        tags_alt = pag_bs.find_all("span", {"itemprop": "height"})
        for tag_idade in tags_idades:
            idade_jog.append(str(tag_idade.text).replace(" ", "")[12:14])

        for tag_nac in tags_nac:
            nac_jog.append(str(tag_nac.text).replace(" ", "")[1:])

        for tag_alt in tags_alt:
            alt_jog.append(float(str(tag_alt.text).replace(",",".")[:5].rstrip()))

    # Players value


    
    dic = {
        'nome':nomes_jog,
        'clube':club_jog,
        'campeonato':camp_jog,
        'posicao':posicoes_jog,
        'idade':idade_jog,
        'nacionalidade':nac_jog,
        'altura':alt_jog
    }

    df = pd.DataFrame(dic)   
    
    return df

In [220]:
df = creating_club_dataframe('cuiaba-ec','28022', headers)
df

,nome,clube,campeonato,posicao,idade,nacionalidade,altura
0,Walter,cuiaba_ec,CampeonatoBrasileiroSérieA,Goleiro,34,Brasil,1.88
1,João Carlos,cuiaba_ec,CampeonatoBrasileiroSérieA,Goleiro,34,Brasil,1.91
2,Vinicius,cuiaba_ec,CampeonatoBrasileiroSérieA,Goleiro,23,Brasil,1.88
3,Alan Empereur,cuiaba_ec,CampeonatoBrasileiroSérieA,Defensor,28,Brasil,1.86
4,Marllon,cuiaba_ec,CampeonatoBrasileiroSérieA,Defensor,30,Brasil,1.86
5,J. Ojeda,cuiaba_ec,CampeonatoBrasileiroSérieA,Defensor,24,Paraguai,1.88
6,Paulão,cuiaba_ec,CampeonatoBrasileiroSérieA,Defensor,36,Brasil,1.87
7,Joaquim,cuiaba_ec,CampeonatoBrasileiroSérieA,Defensor,23,Brasil,1.87
8,Sidcley,cuiaba_ec,CampeonatoBrasileiroSérieA,Defensor,29,Brasil,1.76
9,Igor Cariús,cuiaba_ec,CampeonatoBrasileiroSérieA,Defensor,29,Brasil,1.84


In [209]:
"""
To make the request to the page we have to inform the
website that we are a browser and that is why we
use the headers variable
"""
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

# endereco_da_pagina stands for the data page address
endereco_da_pagina = "https://www.transfermarkt.com.br/america-mineiro/startseite/verein/2863?saison_id=2021"

# In the objeto_response variable we will the download of the web page
objeto_response = requests.get(endereco_da_pagina, headers=headers)

"""
Now we will create a BeautifulSoup object from our object_response.
The 'html.parser' parameter represents which parser we will use when creating our object,
a parser is a software responsible for converting an entry to a data structure.
"""
pagina_bs = BeautifulSoup(objeto_response.content, 'html.parser')

In [216]:
club_jog = [] # List that will receive all the players club - 

transfmkt_site = 'https://www.transfermarkt.com.br'

# Players names
nomes_jog = [] # List that will receive all the players names - check
tags_nomes = pagina_bs.find_all("span", {"class": "show-for-small"})
for tag_nome in tags_nomes:
    nomes_jog.append(tag_nome.text)

# Players links
links_jog = [] # List that will receive all the players profile links - check
tags_links = BeautifulSoup(str(tags_nomes).strip('[]'), 'html.parser')
for a in tags_links.find_all('a', href=True):
    links_jog.append(a['href'])

# Clubs championship
camp_jog = []
tags_camp = pagina_bs.find_all("span", {"class": "hauptpunkt"})
for tag_camp in tags_camp:
    camp = str(tag_camp.text).replace(" ", "").replace("\n","")

for c in range(len(nomes_jog)):
    camp_jog.append(camp)

# Players Positions
posicoes_jog = [] # List that will receive all the players positions - check
tags_goleiros = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Torwart"})
tags_zagueiros = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Abwehr"})
tags_meias = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Mittelfeld"})
tags_atacantes = pagina_bs.find_all("td", {"class": "zentriert rueckennummer bg_Sturm"})

# Gk positions
for c in range(len(tags_goleiros)):
    posicoes_jog.append("Goleiro")

# Defenders positions
for c in range(len(tags_zagueiros)):
    posicoes_jog.append("Defensor")

# Midfielders positions
for c in range(len(tags_meias)):
    posicoes_jog.append("Meia")

# Strikers positions
for c in range(len(tags_atacantes)):
    posicoes_jog.append("Atacante")

# # Players age, heights, nacionalities
# idade_jog = [] # List that will receive all the players age - check
# alt_jog = [] # List that will receive all the players height - check
# nac_jog = [] # List that will receive all the players nacionalities - check

# for link in links_jog:
#     end = transfmkt_site + link
#     obj_resp = requests.get(end, headers=headers)
#     pag_bs = BeautifulSoup(obj_resp.content, 'html.parser')
#     tags_idades = pag_bs.find_all("span", {"itemprop": "birthDate"})
#     tags_nac = pag_bs.find_all("span", {"itemprop": "nationality"})
#     tags_alt = pag_bs.find_all("span", {"itemprop": "height"})
#     for tag_idade in tags_idades:
#         idade_jog.append(str(tag_idade.text).replace(" ", "")[12:14])

#     for tag_nac in tags_nac:
#         nac_jog.append(str(tag_nac.text).replace(" ", "")[1:])

#     for tag_alt in tags_alt:
#         alt_jog.append(float(str(tag_alt.text).replace(",",".")[:5].rstrip()))


In [218]:
# print(nomes_jog, len(nomes_jog))
# print(posicoes_jog, len(posicoes_jog))
# print(idade_jog, len(idade_jog))
# print(nac_jog, len(nac_jog))
# print(alt_jog, len(alt_jog))
print(camp_jog)
len(camp_jog)

['CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA', 'CampeonatoBrasileiroSérieA']


31